<a href="https://colab.research.google.com/github/NeurologiaLogic/Artificial-intelligence/blob/main/deepfashion_clothes_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Num GPUs Available:  1
1 Physical GPUs, 1 Logical GPUs


In [ ]:
#import data
!pip install --upgrade --no-cache-dir gdown
#images
!gdown 1U9YVXRJSYeoW1Jjxw_9CBI0BQUoWVbke
#information
!gdown 1sXKaWanOFuRzCYrodqYjmBTzCzh35FbY

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
Downloading...
From: https://drive.google.com/uc?id=1U9YVXRJSYeoW1Jjxw_9CBI0BQUoWVbke
To: /content/img.zip
100% 2.68G/2.68G [00:39<00:00, 67.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1sXKaWanOFuRzCYrodqYjmBTzCzh35FbY
To: /content/information.zip
100% 18.1M/18.1M [00:00<00:00, 79.4MB/s]


In [ ]:
!unzip information.zip
!unzip img

Streaming output truncated to the last 5000 lines.
  inflating: img/Woven_Dolphin_Shorts/img_00000027.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000028.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000029.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000030.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000031.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000032.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000033.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000034.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000035.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000036.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000037.jpg  
   creating: img/Woven_Drawstring_Joggers/
  inflating: img/Woven_Drawstring_Joggers/img_00000001.jpg  
  inflating: img/Woven_Drawstring_Joggers/img_00000002.jpg  
  inflating: img/Woven_Drawstring_Joggers/img_00000003.jpg  
  inflating: img/Woven_Drawstring_Joggers/img_00000004.jpg  
  inflating: img/Woven_Drawstring_J

In [ ]:
#data provided
abs_path = './information/Anno_coarse'
dataset_path = os.path.join(abs_path,"list_category_img.txt")
df = pd.read_table(dataset_path, sep="\t")
df[['img_path','category_label']] = df['image_name  category_label'].str.split(expand=True)
X = df["img_path"]
y= df["category_label"]
print(y.nunique())

46


In [ ]:
#model structure
from tensorflow.keras.layers import Flatten,Dense,BatchNormalization,Dropout
from tensorflow.keras.applications import EfficientNetV2M
from tensorflow.keras import Model

#base model
base_model = EfficientNetV2M(
    include_top=False,
    weights="imagenet",
    input_shape=(300,300,3)
)

#example
# inputs = tf.keras.Input(shape=(3,))
# x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
# outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)
# model = tf.keras.Model(inputs=inputs, outputs=outputs)

base_model_input = base_model.input
base_model_output = base_model.output
m = Flatten()(base_model_output)
m = Dense(200,activation=tf.nn.relu,name="D1")(m)
m = BatchNormalization()(m)
m = Dropout(0.01)(m)
m = Dense(100,activation=tf.nn.relu,name="D2")(m)
final_output_layer = Dense(46,activation=tf.nn.softmax)(m)
model = Model(inputs=base_model_input,outputs=final_output_layer)
model.summary()

Streaming output truncated to the last 5000 lines.
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op TruncatedNormal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Exe

In [ ]:
#freeze training
for model_struct in model.layers:
  if model_struct.name == "block7e_project_bn":
    break
  model_struct.trainable = False

In [ ]:
#checking trainable params after tweak
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 300, 300, 3)  0           ['input_1[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 150, 150, 24  648         ['rescaling[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
#compiling model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(),)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0


In [ ]:
# datasets split -> too big need split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,shuffle=True,random_state=42)


In [ ]:
def save_batch(start,range):
  X_train_batch = X_train[start:start+range]
  y_train_batch = y_train[start:start+range]
  X_test_batch = X_test[start:start+range]
  y_test_batch = y_train[start:start+range]
  #spliting data into batches
  with open(f'X_train_batch{start}.txt', 'wb') as f1:
    pickle.dump(X_train_batch, f1)
  with open(f'y_train_batch{start}.txt', 'wb') as f2:
    pickle.dump(y_train_batch, f2)
  with open(f'X_test_batch{start}.txt', 'wb') as f3:
    pickle.dump(X_test_batch, f3)
  with open(f'y_test_batch{start}.txt', 'wb') as f4:
    pickle.dump(y_test_batch, f4)

In [ ]:
#dividing the training data into batches
import pickle
size = X_train.size
batch = 100 #50 batch
sizePerBatch = round(size/batch) #ex size of dataset 200k/50 batch
X_train_batch = [0]*sizePerBatch
y_train_batch = [0]*sizePerBatch
X_test_batch = [0]*sizePerBatch
y_test_batch = [0]*sizePerBatch

for i in range(0,size,sizePerBatch):
  save_batch(i,i+sizePerBatch)
  

In [ ]:
#reshaping function 
from PIL import Image
from tensorflow.keras.utils import load_img
from numpy import asarray

def reshaping(img_path):
  return asarray(tf.keras.utils.load_img(img_path).resize((300,300)))

In [ ]:
def get_batch_data(index):
    X_train_batch = open (f"X_train_batch{index}.txt", "rb")
    X_train_batch = pickle.load(X_train_batch)
    y_train_batch = open (f"y_train_batch{index}.txt", "rb")
    y_train_batch = pickle.load(y_train_batch)
    
    X_test_batch = open (f"X_test_batch{index}.txt", "rb")
    X_test_batch = pickle.load(X_test_batch)
    y_test_batch = open (f"y_test_batch{index}.txt", "rb")
    y_test_batch = pickle.load(y_test_batch)
    return X_train_batch,X_test_batch,y_train_batch,y_test_batch

In [ ]:
#training model with batches
checkpoint_path = "training_progress/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

for i in range(0,size,sizePerBatch):
    #loading batches
    X_train_batch,X_test_batch,y_train_batch,y_test_batch = get_batch_data(i)
    
    #preprocessing the input batches
    X_train_batch = [reshaping(image) for image in X_train_batch]
    X_test_batch = [reshaping(image) for image in X_test_batch]

    #training the model
    model.fit(X_train_batch,y_train_batch,
              validation_data=(X_test_batch,y_test_batch),
              callbacks=[cp_callback])
    
    #example
    # model.fit(train_images, 
    #       train_labels,  
    #       epochs=10,
    #       validation_data=(test_images, test_labels),
    #       callbacks=[cp_callback])

NameError: ignored

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
#Predict test


In [ ]:
#test reshaping images
# print(X_train[150])
# from PIL import Image
# from numpy import asarray

# img = asarray(Image.open('/content/img/1981_Graphic_Ringer_Tee/img_00000001.jpg'))
# print(img.shape,img.size)
# img = asarray(Image.open('/content/img/1981_Graphic_Ringer_Tee/img_00000001.jpg').resize((300,300)))
# print(img.shape,img.size)

In [ ]:
#Evaluate Model
from sklearn.metrics import accuracy_score
predicted = [model.predict(x_test) for x_test in X_test]
# Evaluating the model
acc_naive = accuracy_score(y_train,predicted)
print(f'model has {acc_naive.round(2)*100}% accuracy')

In [ ]:
#Save model
import joblib
joblib.dump(model,"deep_fashion_model.pkl")